Sometimes there is an error when trying to create the stim_log using the "make_stim_csv=True" flag. This usually arises if there is some issue with the sweep/sync pulses. Run the code below piece by piece to find the error, it is the same as the repository, but but allows you to interrupt the flow to correct an issue. This will NOT alter the .sync file in any way.

In [1]:
import io
import sys
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import copy
import time

In [2]:
sys.path.append(r'C:\Users\lesliec\code')

In [3]:
from tbd_eeg.tbd_eeg.data_analysis.eegutils import EEGexp

In [4]:
%matplotlib notebook

## EEGexp class

In [5]:
rec_folder = r'V:\mouse730911\aw_stim_train_2024-05-03_10-33-23\experiment1\recording1'
exp = EEGexp(rec_folder, preprocess=False, make_stim_csv=False)

Experiment type: electrical stimulation


In [6]:
print(exp.mouse)
print(exp.experiment_type)
print(exp.stim_instruction_files)
print(exp.stimulus_log_file)

730911
electrical stimulation
{'sensory': [], 'electrical': ['V:\\mouse730911\\aw_stim_train_2024-05-03_10-33-23\\experiment1\\recording1\\stimulation_protocol_1pulse_50tr_25uA_6.5-7.5_ISI.txt', 'V:\\mouse730911\\aw_stim_train_2024-05-03_10-33-23\\experiment1\\recording1\\stimulation_protocol_2pulse_50tr_25uA_6.5-7.5_ISI.txt'], 'opto': []}
V:\mouse730911\aw_stim_train_2024-05-03_10-33-23\experiment1\recording1\estim_log.csv


In [7]:
sync_data = exp._load_sync_dataset()
print(sync_data.line_labels)

['barcodes', 'opto_sync', 'frames', 'behavior_sweep', 'photodiode', 'sweep', '', 'behavior_vsync', 'eyetracking', 'behavior', 'rotA', 'rotB', 'estim_sync', 'estim_sweep', '', '', '', 'opto_sweep', 'opto_trial', '', '', '', '', '', '', '', '', '', '', '', '', '']


# Electrical stim

In [8]:
esweep_rising = sync_data.get_edges(keys=('estim_sweep',), kind='rising', units='seconds')
esweep_falling = sync_data.get_edges(keys=('estim_sweep',), kind='falling', units='seconds')
esync_rising = sync_data.get_edges(keys=('estim_sync',), kind='rising', units='seconds')
esync_falling = sync_data.get_edges(keys=('estim_sync',), kind='falling', units='seconds')

Need to check the sweep signal?

**If there is an extra sweep, identify the index and delete it from both arrays below.**

**This will create a fake sweep signal only if there are no sweeps found**

In [9]:
total_trials = len(esync_rising)
print('Total e-stim trials: %d' % total_trials)
num_sweeps = len(esweep_rising)
print('Number of e-stim sweeps: %d' % num_sweeps)
trials_per_sweep = int(total_trials / num_sweeps)
print('Trials per sweep: %d' % trials_per_sweep)

Total e-stim trials: 720
Number of e-stim sweeps: 3
Trials per sweep: 240


In [10]:
with open(exp.stim_instruction_files['electrical'][0]) as file:
    estim_txt = file.read() # open estim file and read in text
estim = estim_txt.split('channel:') # split and read txt file
d = {}
for item in estim:
    if 'value' in item:
        key = 'channel'+str(item[1])
        d[key]=item.split('\n')
channel1 = d['channel1'][1:]
ch1_df = pd.read_csv(io.StringIO('\n'.join(channel1)), delim_whitespace=True)

In [11]:
list_stim_type = []
list_stim_amp = []
list_stim_dur = []
for i in range(1, len(ch1_df)):
    if ch1_df.value.iloc[i-1] < 0 and ch1_df.value.iloc[i] > 0:
        list_stim_type.append('biphasic')
        list_stim_amp.append(ch1_df.value.iloc[i])
        list_stim_dur.append(ch1_df.time.iloc[i-1] + ch1_df.time.iloc[i])
estim_log = pd.DataFrame({
    'stim_type': list_stim_type,
    'parameter': list_stim_amp,
    'duration': np.array(list_stim_dur) * 1e-6
})

In [12]:
print(len(estim_log))
estim_log.head()

240


,stim_type,parameter,duration
0,biphasic,70,0.0004
1,biphasic,70,0.0004
2,biphasic,50,0.0004
3,biphasic,50,0.0004
4,biphasic,30,0.0004


In [13]:
if len(estim_log) == trials_per_sweep:
    estim_log = pd.concat([estim_log] * num_sweeps, axis='index', ignore_index=True)
else:
    print('estim_log does not match total_trials / num_sweeps')

**This fixed stim log for 546655, when we stopped in the middle of a block.**

In [14]:
estim_log['onset'] = esync_rising
estim_log['offset'] = esync_falling

In [15]:
estim_log['sweep'] = np.zeros(len(estim_log), dtype=int) - 1
for i in range(len(esweep_rising)):
    # print('Sweep %d: %f to %f' % (i, sweep_start, sweep_end))
    estim_log.loc[(estim_log['onset'] >= esweep_rising[i]-5) & (estim_log['onset'] <= esweep_falling[i]+2), 'sweep'] = i

In [16]:
estim_log.head()

,stim_type,parameter,duration,onset,offset,sweep
0,biphasic,70,0.0004,198.68670,198.68710,0
1,biphasic,70,0.0004,205.83477,205.83517,0
2,biphasic,50,0.0004,213.01424,213.01464,0
3,biphasic,50,0.0004,220.15047,220.15087,0
4,biphasic,30,0.0004,227.59610,227.59650,0


In [17]:
estim_log[115:125]

,stim_type,parameter,duration,onset,offset,sweep
115,biphasic,70,0.0004,1006.06239,1006.06279,0
116,biphasic,30,0.0004,1013.05802,1013.05842,0
117,biphasic,50,0.0004,1019.74818,1019.74858,0
118,biphasic,30,0.0004,1026.74363,1026.74403,0
119,biphasic,30,0.0004,1033.39167,1033.39207,0
120,biphasic,50,0.0004,1039.94710,1039.94750,0
121,biphasic,30,0.0004,1047.29827,1047.29867,0
122,biphasic,30,0.0004,1054.35928,1054.35968,0
123,biphasic,30,0.0004,1061.78933,1061.78973,0
124,biphasic,30,0.0004,1068.98644,1068.98684,0


# If just estim, stop here and go to "Combine all logs" step below

## Sensory stim

This is from NP4 with a digital photodiode.<br>Load behavior_sweep, behavior_sync, and photodiode rising and falling edges.

In [8]:
# behavior_sweep
sweep_rising = sync_data.get_edges(keys=('behavior_sweep',), kind='rising', units='seconds')
sweep_falling = sync_data.get_edges(keys=('behavior_sweep',), kind='falling', units='seconds')
# behavior_vsync
vsync_rising = sync_data.get_edges(keys=('behavior_vsync',), kind='rising', units='seconds')
vsync_falling = sync_data.get_edges(keys=('behavior_vsync',), kind='falling', units='seconds')
# photodiode
pd_rising = sync_data.get_edges(keys=('photodiode',), kind='rising', units='seconds')
pd_falling = sync_data.get_edges(keys=('photodiode',), kind='falling', units='seconds')

In [9]:
if len(sweep_rising) == 0:
    between_sweep_inds = np.argwhere(np.diff(vsync_rising) > 10.).flatten()
    sweep_rising = np.concatenate(([vsync_rising[0]-1.], vsync_rising[between_sweep_inds]+2.))
    sweep_falling = np.concatenate((vsync_rising[between_sweep_inds]+1., [vsync_rising[-1]+1.]))

In [9]:
print('behavior_sweep info')
print('number of rising/falling: %d/%d' % (len(sweep_rising), len(sweep_falling)))
print('first rising: %f' % sweep_rising[0])
print('first falling: %f' % sweep_falling[0])

behavior_sweep info
number of rising/falling: 2/2
first rising: 157.130560
first falling: 763.825500


In [10]:
print('behavior_vsync info')
print('number of rising/falling: %d/%d' % (len(vsync_rising), len(vsync_falling)))
print('first rising: %f' % vsync_rising[0])
print('first falling: %f' % vsync_falling[0])

behavior_vsync info
number of rising/falling: 61794/61794
first rising: 157.131140
first falling: 157.132720


In [11]:
print('photodiode info')
print('number of rising/falling: %d/%d' % (len(pd_rising), len(pd_falling)))
print('first rising: %f' % pd_rising[0])
print('first falling: %f' % pd_falling[0])

photodiode info
number of rising/falling: 240/240
first rising: 211.865310
first falling: 212.097540


Load 'timestamp'_stimulus_log.csv

In [17]:
log_long = pd.read_csv(exp.stim_instruction_files['sensory'][0])
print(len(log_long))
log_long.head()

61338


,time,frame,display_stim,stim_type,stim_parameter
0,0.004293,0,False,gray_screen,none
1,0.017292,1,False,gray_screen,none
2,0.020625,2,False,gray_screen,none
3,0.037331,3,False,gray_screen,none
4,0.054198,4,False,gray_screen,none


Remove "gray" screens from log

In [18]:
## NOW remove reliance on vsync pulses ##
log_long = log_long.drop(labels='time', axis='columns')

## Remove gray screen from log ##
visual_stimuli = ['fullscreen', 'circle', 'natural_scene']
list_stim_type = []
list_stim_param = []
for i in range(1, len(log_long)):
    if log_long.stim_type.iloc[i-1] == 'gray_screen' and log_long.stim_type.iloc[i] in visual_stimuli:
        list_stim_type.append(log_long.stim_type.iloc[i])
        list_stim_param.append(log_long.stim_parameter.iloc[i]) # to make sure order matches stim table
stim_log = pd.DataFrame({
    'stim_type': list_stim_type,
    'parameter': list_stim_param,
})

In [15]:
print(pd_rising[:5])

[191.97224 195.98945 199.92549 204.12917 208.16565]


In [15]:
print(len(pd_rising))

240


In [19]:
## These are not the correct onset/offset times yet
print(len(stim_log))
stim_log.head()

240


,stim_type,parameter
0,natural_scene,33
1,natural_scene,65
2,natural_scene,88
3,natural_scene,51
4,natural_scene,114


Correct onset and offset times using photodiode rising and falling times

In [20]:
stim_log['onset'] = pd_rising
stim_log['offset'] = pd_falling
stim_log['duration'] = stim_log['offset'] - stim_log['onset']
stim_log.head()

,stim_type,parameter,onset,offset,duration
0,natural_scene,33,211.86531,212.09754,0.23223
1,natural_scene,65,215.33486,215.58380,0.24894
2,natural_scene,88,218.33738,219.07006,0.73268
3,natural_scene,51,226.67768,226.85989,0.18221
4,natural_scene,114,229.94706,230.21266,0.26560


In [21]:
stim_log['sweep'] = np.zeros(len(stim_log), dtype=int) - 1
for i in range(len(sweep_rising)):
    # print('Sweep %d: %f to %f' % (i, sweep_start, sweep_end))
    stim_log.loc[(stim_log['onset'] > sweep_rising[i]) & (stim_log['onset'] < sweep_falling[i]), 'sweep'] = i

In [22]:
stim_log[0:28]

,stim_type,parameter,onset,offset,duration,sweep
0,natural_scene,33,211.86531,212.09754,0.23223,0
1,natural_scene,65,215.33486,215.58380,0.24894,0
2,natural_scene,88,218.33738,219.07006,0.73268,0
3,natural_scene,51,226.67768,226.85989,0.18221,0
4,natural_scene,114,229.94706,230.21266,0.26560,0
5,natural_scene,103,235.91871,236.15097,0.23226,0
6,natural_scene,117,238.87116,239.15347,0.28231,0
7,natural_scene,116,241.64014,242.38950,0.74936,0
8,natural_scene,82,248.12890,248.86155,0.73265,0
9,natural_scene,40,251.98211,252.74813,0.76602,0


## Combine all logs

In [23]:
all_logs_list = [stim_log] # estim is called estim_log and visual is called stim_log

In [24]:
all_stim_log = pd.concat(all_logs_list, axis=0, sort=False).sort_values(by='onset', axis=0)

In [25]:
all_stim_log.head()

,stim_type,parameter,onset,offset,duration,sweep
0,natural_scene,33,211.86531,212.09754,0.23223,0
1,natural_scene,65,215.33486,215.58380,0.24894,0
2,natural_scene,88,218.33738,219.07006,0.73268,0
3,natural_scene,51,226.67768,226.85989,0.18221,0
4,natural_scene,114,229.94706,230.21266,0.26560,0


## Save the file

In [26]:
all_stim_log.to_csv(exp.stimulus_log_file, index=False)

#### Load file to check?